In [20]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_KEY")

# 데이터베이스 로드 함수
def load_vectorstore(persist_directory):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
    return Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# 검색 함수
def search(query, vectorstore, n_results=1):
    """
    쿼리에 대한 유사성 검색을 수행하고 결과를 반환합니다.

    Args:
        query (str): 검색할 쿼리
        vectorstore (Chroma): Chroma 벡터 저장소 객체
        n_results (int): 반환할 결과 수

    Returns:
        list: 검색된 문서들의 리스트. 각 문서는 'content'와 'metadata'를 포함.
    """
    # 유사성 검색 수행
    docs = vectorstore.similarity_search(query, k=n_results)

    # 검색된 문서에서 필요한 정보 추출
    results = []
    for doc in docs:
        results.append({
            "content": doc.page_content,  # 문서의 내용
            "metadata": doc.metadata      # 문서의 메타데이터
        })

    return results  # 검색된 문서들의 리스트 반환

In [21]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 불러오기
OPENAI_KEY = os.getenv("OPENAI_KEY")

def clear_query(QUERY):
    # OpenAI 클라이언트 초기화
    client = OpenAI(api_key=OPENAI_KEY)

    context = """
당신은 문장을 간결하게 요약하여 핵심적인 의미만 추출하는 전문가입니다. 
주어진 문장에서 불필요한 단어를 제거하고 중요한 내용만 남겨주세요. 
결과는 단순하고 명료한 문장이어야 합니다.

[Examples]
case 1:
- Query: "나 오늘 마라탕 먹을거야!"
- Output: "마라탕 먹다"

case 2:
- Query: "나 오늘 기분이 별로 안 좋아.."
- Output: "오늘 기분 안 좋다"

case 3:
- Query: "오늘 날씨 진짜 좋다!"
- Output: "오늘 날씨 좋다"

[Query]
{query}

[Output]
    """

    functions = [
        {
            "name": "clear_query",
            "description": "사용자의 입력된 문장을 간결하고 핵심적인 의미로 변환하는 함수입니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "사용자로부터 입력된 원래 문장입니다."
                    },
                    "answer": {
                        "type": "string",
                        "description": "입력된 문장을 간결하게 요약하여 핵심적인 의미로 변환한 결과입니다."
                    }
                },
                "required": ["query", "answer"]
            }
        }
    ]

    try:
        # 쿼리 요청
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context.format(query=QUERY)}
            ],
            tools=[{
                "type": "function",
                "function": functions[0]
            }],
            tool_choice={"type": "function", "function": {"name": "clear_query"}}
        )

        # 응답 추출
        function_call = response.choices[0].message.tool_calls[0].function
        function_args = json.loads(function_call.arguments)

        answer = function_args['answer']
        return answer

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return None

In [22]:
result = clear_query("나 오늘 대학 친구들 만나러가!")
print(result)

대학 친구들 만나러 가다


In [24]:
persist_directory = "./chroma_db"

try:
    # 벡터 저장소 로드
    vectorstore = load_vectorstore(persist_directory)
    print("벡터 저장소 로드 완료.")

    # 검색 수행
    query = input("query를 입력하세요: ")
    final_query = clear_query(query)
    print(final_query)
    results = search(final_query, vectorstore, n_results=3)

    # 검색 결과 출력
    for idx, result in enumerate(results, start=1):
        print(f"결과 {idx}:")
        print(f"내용: {result['content']}")
        print(f"메타데이터: {result['metadata']}")
        print("-" * 50)

except Exception as e:
    print(f"오류 발생: {str(e)}")

벡터 저장소 로드 완료.
아침으로 토스트 먹다
결과 1:
내용: - 2025년 1월 11일, 사용자는 마라탕을 먹었다고 전했다. 토핑으로는 푸주를 선택했다. 또한, 딸기 탕후루를 먹었다고 언급했다.이어서 딸기가 들어간 빵을 사먹었다고 말했다. 그 빵에는 크림과 딸기 모카번이 들어있다고 말했다.
메타데이터: {'date': '2025-01-11', 'topic': '음식'}
--------------------------------------------------
결과 2:
내용: - 사용자는 2025년 1월 13일 점심에 '선재 업고 튀어라'라는 드라마를 보면서 엄마가 싸준 마라탕 도시락을 먹을 계획이라고 밝혔습니다. 사용자는 매운 음식을 아주 좋아하며, 특히 떡볶이와 짬뽕이 떠오르는 매운 음식이라고 언급하였습니다. 또한, 주말에도 마라탕을 먹을 계획이라고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '음식'}
--------------------------------------------------
결과 3:
내용: - 사용자는 2025년 1월 12일에 매우 기분이 좋았고, 그냥 좋은 기분이어서 특별한 이유는 없다고 했다.
- 사용자는 맛있는 음식을 먹는 것을 좋아한다. 그날 딸기가 들어간 크림 모카번을 먹었는데, 커피 대신 빵과 함께 먹었다.
- 사용자는 소금빵도 좋아해하며, 마지막으로 먹은 것은 몇 주 전이었다. 그리고 학교 근처의 브레덴코에서 다음에 또 먹을 계획이 있다.
메타데이터: {'date': '2025-01-12', 'topic': '음식'}
--------------------------------------------------
